# PPE needs estimation from confirmed cases at the county level



As the number of covid cases in the states continues to grow, U.S. hospitals are already reporting shortages of key equipment needed to care for critically ill patients, including ventilators and personal protective equipment (PPE) for medical staff. 

![convid19_patient_flow](https://user-images.githubusercontent.com/14057932/77640828-695cee00-6f31-11ea-812b-55c33f2e9de4.png)

*Fig 1* PPE needs through confirmed convid19 case. 


### PPEs in this case include:
- Convid19 virus testkit;
- Droplet mask/surgical mask;
- N95 mask
- Gloves
- Gown
- Faceshields
- Googles
- BP cuff, stethoscope, pulse Ox, Ambu bug
- Sani-wipers
- Ventilators


## Input

**The input data is the confirmed covid19 cases at county level** (Data from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). We load this data when the notebook runs from [this url](https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv) using `covidcaremap.cases.get_usafacts_cases_by_county`. Note that running this notebook on different days will produce different results as it will use the latest data. We took in the newly added case daily for the PPE needs estimation. 

The PPEs are consumed through the covid19 patient flow that has shown in the above diagram. 

In order to make the estimation, we assumed:

- 14% test cases are tested as positive. This can change from county to county. 
- 80% of the confirmed positive cases will have very mild symptoms and only need home care; while
- 20% of the confirmed cases will end up in in-patient care; 
- 40% of patients from in-patient care may need critical care. 

- A patient may have about ten-room visits from doctors and nurses per day in in-patient care; but 
- A patient in critical care may need a room visit every 20 - 30 mins from doctors, nurses, other room visits may come from therapists, ventilator technicians. We assume 40 room visits per day for a critical care patient.

- Each patient need a ventilator in critical care;

- A patient may spend seven days in in-patient care;
- A patient may spend an average of 10 days at critical care. 



## Output

**County-level total PPE needs estimation from the current confirmed case from Jan 22nd to March 24th, 2020**

In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd
import geopandas as gpd

from covidcaremap.cases import get_usafacts_cases_by_county

In [ ]:
#only to use this geojson file because it has the county level geo-polygon
pop_geo = "../../data/processed/us_counties_with_pop.geojson"

In [ ]:
# setting at one, so we can read in confirmed case daily from multiple sources
# in this workthough, we used data from https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

confirmed_case = 1

# one health provider testing 10 patient perday
testing_ratio = 10

positive_rate = 0.14

inpatient_rate = 0.2
homecare_rate = 1- inpatient_rate
test_cases = round(confirmed_case / positive_rate)


critical_rate = 0.2
support_care = 0.6

inpatient_care_days = 7
critical_care_days = 10

## patient to lab technician and healthcare workers
patient_tech_ratio = 175 


# PPE use per patients per day from in patient to critical care
haveay_ppe_use_inp = 10
median_ppe_use_inp = 4 

haveay_ppe_use_icu = 40
median_ppe_use_icu = 6 
light_ppe_use = 1
double = 2


In [ ]:
def ppes_ed_community(patients):
    """compute PPEs include (N95, gloves, gown, testkit, face shield and ventilator) 
    base on patients and healthcareworkers’ cosupution in ED or community testing 
    
    Args:
        patients (int): number of comfirmed cases

    Returns:
        total_ppes_ed_comm (dictionary): PPE needs for ED presentation or community testing
    """
    test_cases = round(confirmed_case / positive_rate)
    healthcareworkers = test_cases/testing_ratio
    droplet_masks = test_cases 
    n95_needs =  healthcareworkers
    gloves = healthcareworkers
    testkits =  test_cases
    gowns =  healthcareworkers
    faceshields = healthcareworkers
    total_ppes_ed_comm = dict(N95s= n95_needs, Droplet_Masks = droplet_masks, Gloves =gloves, Testkits = testkits, Gowns=gowns,
                         Faceshields=faceshields)
    
    return total_ppes_ed_comm


def ppes_lab_processing(patients): 
    """to compute the ppes needs during the lab processing
    Args:
        patients (int): number of total comfirmed cases from community test and ED presentation;
        
    Returns:
        total_ppes_lab (dictionary): PPE needs for lab processing
    """
    test_cases = round(confirmed_case / positive_rate)
    technicians = test_cases/patient_tech_ratio # each technician can run 175 test per day. 
    n95_needs = technicians
    gloves =  technicians
    gowns =  technicians
    faceshields = technicians

    total_ppes_lab = dict(N95s=n95_needs, Gloves= gloves, Gowns=gowns, 
                          Faceshields= faceshields)

    return total_ppes_lab


def ppes_home_care(patients):
    """to compute ppe needs during the homecare
    Args:
        patients (int): number of total patients from community test and ED presentation;
        
    Returns:
        total_ppes_homecare (dictionary): PPE needs for home care
    """

    #20% endup being positive and 80% of these in home care
    home_care_patients = patients * homecare_rate        
    droplet_masks_needs = home_care_patients

    total_ppes_homecare = dict(Droplet_Masks = droplet_masks_needs)
    return total_ppes_homecare


def ppes_in_patient_care(patients, inpatient_care_days):
    """to compute ppe needs during in patient care
    Args:
        patients (int): number of total patients from community test and ED presentation;
        in_patients_days(int): number of days each person will need to be in Patient Care
        
    Returns:
        total_ppes_in_patient (dictionary): PPE needs for in patient care
    """

    # 20% endup being positive and 20% of these in patient care
    in_patient_patients = patients * inpatient_rate
    in_patient_days = in_patient_patients * inpatient_care_days
    
    n95_needs =  in_patient_days* haveay_ppe_use_inp
    gloves = in_patient_days * haveay_ppe_use_inp * double # 2x of 40 per patients 
    gowns = in_patient_days * haveay_ppe_use_inp # 40 gowns per paitient
    faceshields = in_patient_days * haveay_ppe_use_inp 
    goggles =  in_patient_days * median_ppe_use_inp # goggle can be reused
    bp_cuff = in_patient_days * light_ppe_use # bp cuff, ambu bag can be reused
    wipers = in_patient_days * light_ppe_use

    total_ppes_in_patient = dict(N95s = n95_needs, Gloves= gloves, Gowns=gowns, 
                                 Faceshields= faceshields, Goggles = goggles, 
                                 BP_cuff = bp_cuff, Wiper = wipers)

    return total_ppes_in_patient


def ppes_critical_care(patients, critical_care_days):
    """compute the ventilators and ppe needs based on the critical care patients
    Args:
        patients (int): number of total patients from community test and ED presentation;
        in_patients_days(int): number of days each person will need to be in Critical Care
        
    Returns:
        total_ppes_criticalcare (dictionary): PPE needs for in critical care
    """

    # 40% of in patient patients end up in critical care
    critical_care_patients = patients * positive_rate * critical_rate
    
    critical_patient_days = critical_care_patients * critical_care_days 
    
    ventilators = critical_care_patients

    # assuming 1 doctor takes care the 3 patients over 10 days- needs to check it with Abi 
    n95_needs = critical_patient_days * haveay_ppe_use_icu
    gloves =  critical_patient_days * haveay_ppe_use_icu * double # 2x of 40 per patients 
    gowns = critical_patient_days * haveay_ppe_use_icu # 40 gowns per paitient
    faceshields = critical_patient_days* haveay_ppe_use_icu
    goggles =  critical_patient_days* median_ppe_use_icu # goggle can be reused
    bp_cuff = critical_patient_days * light_ppe_use # bp cuff, ambu bag can be reused
    wipers = critical_patient_days * light_ppe_use
    ventilators = critical_patient_days * light_ppe_use

    total_ppes_criticalcare = dict(N95s = n95_needs, Gloves= gloves, Gowns=gowns, 
                                  Faceshields= faceshields,Goggles = goggles, 
                                  BP_cuff = bp_cuff, Wiper = wipers,
                                  Ventilators = ventilators)
    
    return total_ppes_criticalcare


In [ ]:
# for every confirmed convid19 case
total_ppes_ed_comm = ppes_ed_community(confirmed_case)
total_ppes_ed_comm 

In [ ]:
# for every confirmed convid19 case
total_ppes_lab = ppes_lab_processing(confirmed_case)
total_ppes_lab

In [ ]:
# for every confirmed convid19 case
total_ppes_homecare = ppes_home_care(confirmed_case)
total_ppes_homecare

In [ ]:
# for every confirmed convid19 case
total_ppes_in_patient = ppes_in_patient_care(confirmed_case, inpatient_care_days)
total_ppes_in_patient

In [ ]:
# for every confirmed convid19 case
total_ppes_criticalcare = ppes_critical_care(confirmed_case, critical_care_days)
total_ppes_criticalcare

In [ ]:
# for every confirmed convid19 case from home in patient and critical cares 
testkits = total_ppes_ed_comm['Testkits']
droplet_masks = total_ppes_homecare['Droplet_Masks'] + total_ppes_ed_comm['Droplet_Masks'] 
n95s =  total_ppes_in_patient['N95s'] + total_ppes_criticalcare['N95s'] + total_ppes_ed_comm['N95s'] + total_ppes_lab['N95s'] 
gloves = total_ppes_in_patient['Gloves'] + total_ppes_criticalcare['Gloves']  + total_ppes_lab['Gloves']+ total_ppes_ed_comm['Gloves']
gowns = total_ppes_in_patient['Gowns'] + total_ppes_criticalcare['Gowns'] + total_ppes_lab['Gowns']+ total_ppes_ed_comm['Gowns']
faceshields = total_ppes_in_patient['Faceshields'] + total_ppes_criticalcare['Faceshields'] + total_ppes_lab['Faceshields'] + total_ppes_ed_comm['Faceshields']
goggles = total_ppes_in_patient['Goggles'] + total_ppes_criticalcare['Goggles']
bp_cuffs = total_ppes_in_patient['BP_cuff'] + total_ppes_criticalcare['BP_cuff']
wipers = total_ppes_in_patient['Wiper'] + total_ppes_criticalcare['Wiper']
ventilators = total_ppes_criticalcare['Ventilators']

In [ ]:
total_ppes = round(sum([n95s, gloves, gowns, faceshields,
                        droplet_masks, goggles, bp_cuffs, wipers]))

In [ ]:
# every confirm case may end up using 155 ppes, including n95 masks, gowns, faceshieldd, droplet_masks, googles, bp_cuff, wipers, ventilators
total_ppes

In [ ]:
confirmed_cases = get_usafacts_cases_by_county()
confirmed_cases.head(4)

In [ ]:
confirmed_cases.keys()

In [ ]:
len(confirmed_cases.keys())

In [ ]:
def daily_cases_growth(df, opp_col):
    """ compute daily growth cases from the accumulated cases at county level
    Args:
        df (Pandas dataframe): the accumulated cases from USAFacts;
        opp_col (int):the column where cofirmed case starts to be recorded
    Returns:
        df(Pandas dataframe): dataframe that has daily growth confirmed cases
    """
    col_len = len(df.keys())
    cat_len = opp_col + 1
    # compute the daily growth instead of accumulated cases by county
    df.iloc[:, cat_len:]  = df.iloc[:, -(col_len-cat_len):] - df.shift(axis=1, fill_value=0).iloc[:, -(col_len-cat_len):]
    return df

In [ ]:
def multiple_ppe(ppe_item_total, df_daily, opp_col, item_name):
    """to compute totoal ppes from given dates
    
    Args:
        ppe_item_total (int): the total ppe item from confirm covid19 cases;
        df (Pandas dataframe): data frame that records daily growning confirm cases 
        opp_col (int): the column where cofirmed case starts to be recorded
        
    Returns:
        ppes_df(Pandas dataframe): dataframe has the total sum of the ppe items
    """
    df_n = df_daily.apply(lambda x: x[opp_col:] * ppe_item_total, axis = 1) 
    total_g_ppes = df_n.apply(np.sum, axis =1)
    ppes_df = df_daily.iloc[:,:opp_col]
    # keep the nagative daily convid growth for ventilator compuation 
    if ppe_item_total == ventilators:
        total_g_ppes = np.floor(total_g_ppes)
    # take nagative daily gowth to be 0 for other ppe calculation
    else: 
        df_n= df_n.clip(lower = 0)
        total_g_ppes = df_n.apply(np.sum, axis =1)
    
    #if we see patient in the critical care, we assume at least 1 ventilator is needed
    ppes_df['total_cases'] = df_daily.iloc[:, opp_col:].sum(axis=1)
    ppes_df[item_name] = round(total_g_ppes)
    
    return ppes_df
    

In [ ]:
df_daily_county = daily_cases_growth(confirmed_cases, 4)

In [ ]:
df_daily_county.head()

In [ ]:
total_ppes_df = multiple_ppe(total_ppes, df_daily_county, 4, "total_ppes")
total_ppes_df.head()

In [ ]:
total_testkits = multiple_ppe(testkits, df_daily_county, 4, "testkits")
total_testkits.head()

In [ ]:
total_testkits = multiple_ppe(testkits, confirmed_cases, 4, "testkits")
total_testkits.head()

In [ ]:
total_droplet_masks = multiple_ppe(droplet_masks, df_daily_county, 4, "total_droplet_masks") 
total_droplet_masks.head()

In [ ]:
total_n95s = multiple_ppe(n95s, df_daily_county, 4, "total_n95s") 
total_n95s.head()

In [ ]:
total_gloves = multiple_ppe(gloves, df_daily_county, 4, "total_gloves") 
total_gloves.head()

In [ ]:
total_gowns = multiple_ppe(gowns, df_daily_county, 4, "total_gowns") 
total_gowns.head()

In [ ]:
total_faceshields = multiple_ppe(faceshields, df_daily_county, 4, "total_faceshields") 
total_faceshields.head()

In [ ]:
total_goggles = multiple_ppe(goggles, df_daily_county, 4, "total_goggles") 
total_goggles.head()

In [ ]:
total_bp_cuffs = multiple_ppe(bp_cuffs, confirmed_cases, 4, "total_bp_cuffs") 
total_bp_cuffs.head()

In [ ]:
total_wipers = multiple_ppe(wipers, confirmed_cases, 4, "total_wipers") 
total_wipers.head()

In [ ]:
total_ventilators = multiple_ppe(ventilators, confirmed_cases, 4, "total_ventilators") 
total_ventilators.head()

In [ ]:
total_ppes_df['total_cases'] =total_testkits['total_cases']
total_ppes_df['testkits'] =total_testkits['testkits']
total_ppes_df['total_droplet_masks'] = total_droplet_masks['total_droplet_masks']
total_ppes_df['total_n95s'] = total_n95s['total_n95s']
total_ppes_df['total_gowns'] = total_gowns['total_gowns']
total_ppes_df['total_faceshields'] = total_faceshields['total_faceshields']
total_ppes_df['total_goggles'] = total_goggles['total_goggles']
total_ppes_df['total_bp_cuffs'] = total_bp_cuffs['total_bp_cuffs']
total_ppes_df['total_wipers'] = total_wipers['total_wipers']
total_ppes_df['total_ventilators'] = total_ventilators['total_ventilators']

In [ ]:
total_ppes_df = total_ppes_df.rename(columns={'countyFIPS': 'COUNTY_FIPS'})
total_ppes_df.head()

In [ ]:
gpd_df = gpd.read_file(pop_geo)
gpd_df.head()

In [ ]:
len(gpd_df), len(total_ventilators)

In [ ]:
gpd_df['COUNTY_FIPS'].dtype, total_ppes_df['COUNTY_FIPS'].dtype

In [ ]:
gpd_df['COUNTY_FIPS'] =gpd_df['COUNTY_FIPS'].astype('int64')

In [ ]:
gpd_df_new = gpd_df

In [ ]:
gpd_df_new = gpd_df_new.drop(columns = ['LSAD', 'CENSUSAREA', 'Population'])
gpd_df_new.head()

In [ ]:
total_ppes_geo = gpd_df_new.merge(total_ppes_df, on='COUNTY_FIPS', how='left')

In [ ]:
total_ppes_geo = total_ppes_geo.drop(columns=['County Name_y', 'State_y', 'stateFIPS'])
total_ppes_geo = total_ppes_geo.rename(columns={'County Name_x': 'County Name',
                                               'State_x': 'State'})
total_ppes_geo.head()

In [ ]:
total_ppes_geo.to_file("../../data/processed/ppe_needs_county_level.geojson", driver='GeoJSON')